# Structure from Motion

### The Projective space

Points in $ \mathbb{P}^2 $ are represented as

$$ \mathbf{x} = \begin{bmatrix}
x & y & w
\end{bmatrix}^T
$$

The equivalent point in $ \mathbb{R}^2 $ is

$$ \mathbf{x} = \begin{bmatrix}
x / w & y / w
\end{bmatrix}^T
$$

For points with $w \neq 0$. Points with $w=0$ is called points as infinity.

Lines are represented as

$$
\mathbf{x}^T\mathbf{l} = 0
$$

Where 

$$ \mathbf{l} = \begin{bmatrix}
a & b & c
\end{bmatrix}^T
$$


### Null space

Finding any non-zero vector $\mathbf{x}$ satifying the equation

$$
A\mathbf{x} = \mathbf{0}
$$

The nullspace is only determined up to a scale.

In [1]:
import numpy as np
from numpy.linalg import svd

def nullspace(A, atol=1e-13, rtol=0):
    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

A = np.array([[1, 0],
              [0, 1]])
print(nullspace(A))

A = np.array([[1, 0],
              [0, 0]])
n = nullspace(A)
print(n)
print(A @ n)
k = 17
print(A @ (k * n))


[]
[[0.]
 [1.]]
[[0.]
 [0.]]
[[0.]
 [0.]]


### Projective camera

A projective camera $\texttt{P}$ transforms a world point $\mathbf{X}$ into a image point $\mathbf{x}.$

$$\mathbf{x} = \texttt{P}\mathbf{X}$$

Where $\texttt{P}$ is a 3x4 matrix and the world point $\mathbf{X}$ is a 4-vector while the imgage point $\mathbf{x}$ is a 3-vector.

In [2]:
P = np.array([
    [20,  0, 0, -10],
    [ 0, 20, 0, -10],
    [ 0,  0, 1, 0],
])
X = [3, 4, 10, 1]
x = P @ X
# todo plot this
print(x / x[2])

[5. 7. 1.]


The camera matrix $\texttt{P}$ can be decomposed into the calibration matrix $\texttt{K}$ and a rotation and translation pair like so

$$
\texttt{P} = \texttt{K}
\begin{bmatrix}
\texttt{R} & \mathbf{t}
\end{bmatrix}
$$

Sometimes the camera centre $\tilde{C}$ is needed expicitly

$$
\mathbf{t} = -\texttt{R}\tilde{C}
$$

In [3]:
K = np.array([
    [10, 0, 50],
    [0, 10, 50],
    [0, 0, 1]
])
R = np.eye(3)
t = np.array([10, 10, 0])
P = K @ np.hstack([R, t[:, None]])
print(P)

[[ 10.   0.  50. 100.]
 [  0.  10.  50. 100.]
 [  0.   0.   1.   0.]]


### Calibration matrix

In [274]:
from ipycplot import Plot3D, rectangle

width, height = 320, 200

n = 10
x, y = np.meshgrid(np.linspace(0, width, n), np.linspace(0, height, n))
w = np.ones(x.shape)
# points in the image plane
p1 = np.stack([x, y, w], axis=-1).reshape((-1, 3))

plot = Plot3D()
plot.scale = np.array([200, 200, 1])
target = np.array([width/2, height/2, 0])
plot.camera.lookat(target=target)
def update_focal(f, ay, px, py):
    K = np.array([
        [f, 0, px],
        [0, f, py],
        [0, 0, 1],
    ])
    normals = (np.linalg.inv(K) @ p1.T).T
    p2 = p1 + numgl.normalized(normals.T, axis=0).T * 50
    lines = np.hstack([p1, p2]).reshape(-1, 3)
    
    r = 600
    eye = np.array([r * np.sin(ay), 0, r * np.cos(ay)])
    plot.camera.lookat(eye=eye + target)
    
    plot.canvas.clear()
    plot.lines(lines)
    plot.lines(rectangle(width, height))

interact(update_focal, f=(50, 1000), ay=(-np.pi/2, np.pi/2), px=(0, width), py=(0, height))
plot.show()

interactive(children=(IntSlider(value=525, description='f', max=1000, min=50), FloatSlider(value=0.0, descript…

RoughCanvas()

### Epipolar Geometry
The geometry of two cameras depicting the same scene. 
* The _epipolar point_ is the image of the camera center of the other image. It may lie outside the visible image.
* The _epipolar line_ is the line between the two camera centers.


In [291]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numgl
from ipywidgets import interact
from ipycplot import Plot3D, rectangle, transform


width, height = 320, 200

# world points
n = 10
X = np.random.normal(size=(n, 3), scale=100)


plot = Plot3D()
plot.scale = np.array([80, 80, 1])


right_pose = numgl.translate(( 230, -100, 500)) @ numgl.roty(0.5*np.pi - 1.3)

middle = np.array([0.5, 0.5, 0])

def update(ay, lefta, f):
    #left_pose =  numgl.translate((-200, -100, 500)) @ numgl.roty(lefta)
    left_pose =  numgl.translate((0, 0, 500)) @ numgl.roty(lefta)
    
    center = np.array([0, 150, 0])
    target = center
    r = 1100
    eye = center + np.array([r * np.sin(ay), 0, r * np.cos(ay)]) + np.array([0, -300, 0])
    plot.camera.lookat(eye=eye, target=target)

    plot.canvas.clear()
    plot.lines(transform(left_pose, rectangle(width, height)))
    #plot.lines(transform(right_pose, rectangle(width, height)))
    
    plot.circles(X)
    
    # project points in left camera
    #f = 400
    K = np.array([
        [f, 0, width/2],
        [0, f, height/2],
        [0, 0, 1],
    ])
    R1, C1 = left_pose[0:3, 0:3], left_pose[0:3, 3][:, None]
    t1 = -R1.T @ C1
    P1 = K @ np.hstack([R1, t1])
    
    x1 = transform(P1, X)  # transform world points using camera
    # clip points to image rectangle
    mask = (x1[:, 0] > 0) * (x1[:, 0] < width) * (x1[:, 1] > 0) * (x1[:, 1] < height)
    x1 = x1[mask]
    #print(x1)
    x1 = np.hstack([x1, np.zeros((x1.shape[0], 1))])
    
    plot.circles(transform(left_pose, x1))
    
interact(update, ay=(-np.pi/2, np.pi/2), lefta=(0, 2*np.pi), f=(10, 800))

plot.show()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


interactive(children=(FloatSlider(value=0.0, description='ay', max=1.5707963267948966, min=-1.5707963267948966…

RoughCanvas()

### The Fundamental Matrix
The fundamental matrix maps points in the left image to lines in the right image.

$$
\mathbf{l'} = \texttt{F}\mathbf{x}
$$


$$
\mathbf{x'}^T\texttt{F}\mathbf{x} = \mathbf{0}
$$



### Extracting cameras

### Triangulation

# Images
